## import library

In [1]:
import os
import numpy as np
import tensorflow as tf
from data_loader import *
from data_utils import *
from model import *
from const import *

c:\users\shimd\anaconda3\envs\mondeique\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\shimd\anaconda3\envs\mondeique\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\shimd\anaconda3\envs\mondeique\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\shimd\anaconda3\envs\mondeique\lib\site-packages\ten

# prepare data

In [2]:
train_data, test_data = getBagImage()

Reading csv................
Load bag image...................
Number of bag train data:  1955
Number of bag test data:  516
---------------------------------------------------------------


In [3]:
def one_hot(index, num_classes):
    tmp = np.zeros(num_classes, dtype=np.float32)
    tmp[index] = 1.0
    return tmp

# Main Training code

## Input tensor + Model Network 

In [4]:
sess = tf.InteractiveSession()
global_step = tf.contrib.framework.get_or_create_global_step()

x, y, mask = input_tensor()

y_color_conv, y_shape_conv, y_opening_conv, y_strap_conv, y_pattern_conv, y_carrying_conv, y_charm_conv, y_decoration_conv, is_training, keep_prob = multi_label_net(x)

W0818 16:10:07.418289  7928 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0818 16:10:07.419270  7928 deprecation.py:323] From <ipython-input-4-eaa24b9c7f9c>:2: get_or_create_global_step (from tensorflow.contrib.framework.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.get_or_create_global_step
W0818 16:10:07.424230  7928 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\multi-label-classification\model.py:7: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0818 16:10:07.427222  7928 deprecation_wrapper.py:119] From C:\Users\shimd\

## Loss + mask 씌우기

In [5]:
color_loss, shape_loss, opening_loss, strap_loss, pattern_loss, carrying_loss, charm_loss, decoration_loss, l2_loss, total_loss = selective_loss(y_color_conv, y_shape_conv, y_opening_conv, y_strap_conv, y_pattern_conv, y_carrying_conv, y_charm_conv, y_decoration_conv,y, mask)

train_step = train_op(total_loss, global_step)

color_mask = tf.get_collection('color_mask')[0]
shape_mask = tf.get_collection('shape_mask')[0]
opening_mask = tf.get_collection('opening_mask')[0]
strap_mask = tf.get_collection('strap_mask')[0]
pattern_mask = tf.get_collection('pattern_mask')[0]
carrying_mask = tf.get_collection('carrying_mask')[0]
charm_mask = tf.get_collection('charm_mask')[0]
decoration_mask = tf.get_collection('decoration_mask')[0]

y_color = tf.get_collection('y_color')[0]
y_shape = tf.get_collection('y_shape')[0]
y_opening = tf.get_collection('y_opening')[0]
y_strap = tf.get_collection('y_strap')[0]
y_pattern = tf.get_collection('y_pattern')[0]
y_carrying = tf.get_collection('y_carrying')[0]
y_charm = tf.get_collection('y_charm')[0]
y_decoration = tf.get_collection('y_decoration')[0]

color_correct_prediction = tf.equal(tf.argmax(y_color_conv, 1), tf.argmax(y_color, 1))
shape_correct_prediction = tf.equal(tf.argmax(y_shape_conv, 1), tf.argmax(y_shape, 1))
opening_correct_prediction = tf.equal(tf.argmax(y_opening_conv, 1), tf.argmax(y_opening, 1))
strap_correct_prediction = tf.equal(tf.argmax(y_strap_conv, 1), tf.argmax(y_strap, 1))
pattern_correct_prediction = tf.equal(tf.argmax(y_pattern_conv, 1), tf.argmax(y_pattern, 1))
carrying_correct_prediction = tf.equal(tf.argmax(y_carrying_conv, 1), tf.argmax(y_carrying, 1))
charm_correct_prediction = tf.equal(tf.argmax(y_charm_conv, 1), tf.argmax(y_charm, 1))
decoration_correct_prediction = tf.equal(tf.argmax(y_decoration_conv, 1), tf.argmax(y_decoration, 1))

color_true_pred = tf.reduce_sum(tf.cast(color_correct_prediction, dtype=tf.float32) * color_mask)
shape_true_pred = tf.reduce_sum(tf.cast(shape_correct_prediction, dtype=tf.float32) * shape_mask)
opening_true_pred = tf.reduce_sum(tf.cast(opening_correct_prediction, dtype=tf.float32) * opening_mask)
strap_true_pred = tf.reduce_sum(tf.cast(strap_correct_prediction, dtype=tf.float32) * strap_mask)
pattern_true_pred = tf.reduce_sum(tf.cast(pattern_correct_prediction, dtype=tf.float32) * pattern_mask)
carrying_true_pred = tf.reduce_sum(tf.cast(carrying_correct_prediction, dtype=tf.float32) * carrying_mask)
charm_true_pred = tf.reduce_sum(tf.cast(charm_correct_prediction, dtype=tf.float32) * charm_mask)
decoration_true_pred = tf.reduce_sum(tf.cast(decoration_correct_prediction, dtype=tf.float32) * decoration_mask)

real_train_data = []

# Mask : color -> 0 , shape -> 1, opening -> 2, strap -> 3, pattern -> 4, carrying -> 5, charm -> 6, decoration -> 7
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][0], 16), 0.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][1], 16), 1.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][2], 16), 2.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][3], 16), 3.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][4], 16), 4.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][5], 16), 5.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][6], 16), 6.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][7], 16), 7.0))

saver = tf.train.Saver()

W0818 16:10:08.189017  7928 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\multi-label-classification\model.py:94: The name tf.add_to_collection is deprecated. Please use tf.compat.v1.add_to_collection instead.

W0818 16:10:08.201982  7928 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\multi-label-classification\model.py:122: The name tf.log is deprecated. Please use tf.math.log instead.

W0818 16:10:08.266271  7928 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\multi-label-classification\model.py:158: The name tf.train.exponential_decay is deprecated. Please use tf.compat.v1.train.exponential_decay instead.

W0818 16:10:08.271257  7928 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\multi-label-classification\model.py:159: The name tf.train.MomentumOptimizer is deprecated. Please use tf.compat.v1.train.MomentumOptimizer instead.



## restore model 생성

In [6]:
if not os.path.isfile(SAVE_FOLDER3 + 'model_v3_2.ckpt.index'):
    print('Create new model')
    sess.run(tf.global_variables_initializer())
    print('OK')
else:
    print('Restoring existed model')
    saver.restore(sess, SAVE_FOLDER3 + 'model_v3_2.ckpt')
    print('OK')

loss_summary_placeholder = tf.placeholder(tf.float32)
tf.summary.scalar('loss', loss_summary_placeholder)
merge_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("./summary/")

learning_rate = tf.get_collection('learning_rate')[0]

Create new model
OK


## epoch 에 따라서 training 수행

In [ ]:
current_epoch = (int)(global_step.eval() / (len(real_train_data) // BATCH_SIZE))
for epoch in range(current_epoch + 1, NUM_EPOCHS):
    print('Epoch:', str(epoch))
    np.random.shuffle(real_train_data)
    train_img = []
    train_label = []
    train_mask = []

    for i in range(len(real_train_data)):
        train_img.append(real_train_data[i][0])
        train_label.append(real_train_data[i][1])
        train_mask.append(real_train_data[i][2])

    number_batch = len(real_train_data) // BATCH_SIZE

    avg_ttl = []
    avg_rgl = []
    avg_color_loss = []
    avg_shape_loss = []
    avg_opening_loss = []
    avg_strap_loss = []
    avg_pattern_loss = []
    avg_carrying_loss = []
    avg_charm_loss = []
    avg_decoration_loss = []

    color_nb_true_pred = 0
    shape_nb_true_pred = 0
    opening_nb_true_pred = 0
    strap_nb_true_pred = 0
    pattern_nb_true_pred = 0
    carrying_nb_true_pred = 0
    charm_nb_true_pred = 0
    decoration_nb_true_pred = 0

    color_nb_train = 0
    shape_nb_train = 0
    opening_nb_train = 0
    strap_nb_train = 0
    pattern_nb_train = 0
    carrying_nb_train = 0
    charm_nb_train = 0
    decoration_nb_train = 0

    print("Learning rate: %f" % learning_rate.eval())
    for batch in range(number_batch):
        # print('Training on batch {0}/{1}'.format(str(batch + 1), str(number_batch)))
        top = batch * BATCH_SIZE
        bot = min((batch + 1) * BATCH_SIZE, len(real_train_data))
        batch_img = np.asarray(train_img[top:bot])
        batch_label = np.asarray(train_label[top:bot])
        batch_mask = np.asarray(train_mask[top:bot])
        

        for i in range(BATCH_SIZE):
            if batch_mask[i] == 0.0:
                color_nb_train += 1
            else:
                if batch_mask[i] == 1.0:
                    shape_nb_train += 1
                else:
                    if batch_mask[i] == 2.0:
                        opening_nb_train += 1
                    else:
                        if batch_mask[i] == 3.0:
                            strap_nb_train += 1
                        else:
                            if batch_mask[i] == 4.0:
                                pattern_nb_train += 1
                            else:
                                if batch_mask[i] == 5.0:
                                    carrying_nb_train += 1
                                else:
                                    if batch_mask[i] == 6.0:
                                        charm_nb_train += 1
                                    else:
                                        decoration_nb_train +=1

        batch_img = augmentation(batch_img, 64)

        ttl, colorl, shapel, openingl, strapl, patternl, carryingl, charml, decol, l2l, _ = sess.run([total_loss, color_loss, shape_loss, opening_loss, strap_loss,
                                                                                      pattern_loss, carrying_loss, charm_loss, decoration_loss, l2_loss, train_step],
                                                                                      feed_dict={x: batch_img, y: batch_label, mask: batch_mask, is_training: True, keep_prob: DROP_OUT_PROB})

        color_nb_true_pred += sess.run(color_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                   is_training: True,keep_prob: DROP_OUT_PROB})

        shape_nb_true_pred += sess.run(shape_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                   is_training: True, keep_prob: DROP_OUT_PROB})

        opening_nb_true_pred += sess.run(opening_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                       is_training: True, keep_prob: DROP_OUT_PROB})

        strap_nb_true_pred += sess.run(strap_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                   is_training: True, keep_prob: DROP_OUT_PROB})

        pattern_nb_true_pred += sess.run(pattern_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                       is_training: True, keep_prob: DROP_OUT_PROB})

        carrying_nb_true_pred += sess.run(carrying_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                         is_training: True, keep_prob: DROP_OUT_PROB})

        charm_nb_true_pred += sess.run(charm_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                         is_training: True, keep_prob:DROP_OUT_PROB})

        decoration_nb_true_pred += sess.run(decoration_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                         is_training: True, keep_prob: DROP_OUT_PROB})

        avg_ttl.append(ttl)
        avg_color_loss.append(colorl)
        avg_shape_loss.append(shapel)
        avg_opening_loss.append(openingl)
        avg_strap_loss.append(strapl)
        avg_pattern_loss.append(patternl)
        avg_carrying_loss.append(carryingl)
        avg_charm_loss.append(charml)
        avg_decoration_loss.append(decol)

        avg_rgl.append(l2l)

    color_train_accuracy = color_nb_true_pred * 1.0 / color_nb_train
    shape_train_accuracy = shape_nb_true_pred * 1.0 / shape_nb_train
    opening_train_accuracy = opening_nb_true_pred * 1.0 / opening_nb_train
    strap_train_accuracy = strap_nb_true_pred * 1.0 / strap_nb_train
    pattern_train_accuracy = pattern_nb_true_pred * 1.0 / pattern_nb_train
    carrying_train_accuracy = carrying_nb_true_pred * 1.0 / carrying_nb_train
    charm_train_accuracy = charm_nb_true_pred * 1.0 / charm_nb_train
    decoration_train_accuracy = decoration_nb_true_pred * 1.0 / decoration_nb_train


    #     print('Avg_ttl: ' + str(avg_ttl))
    #     print('loss_summary_placeholder: ' + str(loss_summary_placeholder))
    #     print('merge_summary: ' + str(merge_summary))
    
    # 밑의 두 줄 error 발생해서 주석처리 해놓았음. TODO!
    
    #summary = sess.run(merge_summary, feed_dict={loss_summary_placeholder: avg_ttl})
    #writer.add_summary(summary, global_step=epoch)

    with open('log.csv', 'w+') as f:
        # epochs, color_train_accuracy, shape_train_accuracy, opening_train_accuracy,
        # avg_color_loss, avg_shape_loss, avg_opening_loss, avg_ttl, avg_rgl
        f.write('{0},{1},{2},{3},{4},{5},{6},{7},{8},{9},{10},{11},{12},{13},{14}, {15}, {16}, {17}, {18}\n'.format(current_epoch, color_train_accuracy, shape_train_accuracy, opening_train_accuracy,
                                                                                            strap_train_accuracy, pattern_train_accuracy, carrying_train_accuracy, charm_train_accuracy, decoration_train_accuracy,
                                                                                            avg_color_loss, avg_shape_loss, avg_opening_loss, avg_strap_loss, avg_pattern_loss,
                                                                                            avg_carrying_loss, avg_charm_loss, avg_decoration_loss, avg_ttl, avg_rgl))

    print('color task train accuracy: ' + str(color_train_accuracy * 100))
    print('shape task train accuracy: ' + str(shape_train_accuracy * 100))
    print('opening task train accuracy: ' + str(opening_train_accuracy * 100))
    print('strap task train accuracy: ' + str(strap_train_accuracy * 100))
    print('pattern task train accuracy: ' + str(pattern_train_accuracy * 100))
    print('carrying task train accuracy: ' + str(carrying_train_accuracy * 100))
    print('charm task train accuracy: ' + str(charm_train_accuracy * 100))
    print('decoration task train accuracy: ' + str(decoration_train_accuracy * 100))

#     print('Total loss: ' + str(avg_ttl) + '. L2-loss: ' + str(avg_rgl))
#     print('Color loss: ' + str(avg_color_loss))
#     print('Shape loss: ' + str(avg_shape_loss))
#     print('opening loss: ' + str(avg_opening_loss))
#     print('strap loss: ' + str(avg_strap_loss))
#     print('pattern loss: ' + str(avg_pattern_loss))
#     print('carrying loss: ' + str(avg_carrying_loss))
#     print('charm loss: ' + str(avg_charm_loss))
#     print('decoration loss: ' + str(avg_decoration_loss))

    print('\n')

    saver.save(sess, SAVE_FOLDER3 + 'model_v3_2.ckpt')

Epoch: 1
Learning rate: 0.001000
color task train accuracy: 24.47408927655208
shape task train accuracy: 64.15770609318996
opening task train accuracy: 32.08610968733983
strap task train accuracy: 66.3422131147541
pattern task train accuracy: 65.625
carrying task train accuracy: 44.848795489492566
charm task train accuracy: 45.54759467758444
decoration task train accuracy: 83.46980552712385


Epoch: 2
Learning rate: 0.001000
color task train accuracy: 34.64889800102512
shape task train accuracy: 65.69380440348182
opening task train accuracy: 32.00204813108039
strap task train accuracy: 67.09379805228089
pattern task train accuracy: 86.66666666666667
carrying task train accuracy: 59.682377049180324
charm task train accuracy: 59.754224270353305
decoration task train accuracy: 83.81976446492575


Epoch: 3
Learning rate: 0.001000
color task train accuracy: 36.92307692307693
shape task train accuracy: 65.93237704918032
opening task train accuracy: 31.985670419651996
strap task train accurac

Epoch: 22
Learning rate: 0.000599
color task train accuracy: 66.54713114754098
shape task train accuracy: 76.13927291346646
opening task train accuracy: 31.94871794871795
strap task train accuracy: 72.22792607802874
pattern task train accuracy: 67.02508960573476
carrying task train accuracy: 79.37563971340839
charm task train accuracy: 80.17418032786885
decoration task train accuracy: 83.93039918116683


Epoch: 23
Learning rate: 0.000599
color task train accuracy: 69.51844262295081
shape task train accuracy: 76.88364941055869
opening task train accuracy: 31.967213114754102
strap task train accuracy: 73.2343909928352
pattern task train accuracy: 60.89185033316248
carrying task train accuracy: 81.29164531009738
charm task train accuracy: 81.42272262026611
decoration task train accuracy: 84.26447975397232


Epoch: 24
Learning rate: 0.000569
color task train accuracy: 70.98923628908253
shape task train accuracy: 76.66666666666667
opening task train accuracy: 31.983598154792414
strap task t

Epoch: 43
Learning rate: 0.000358
color task train accuracy: 79.91803278688525
shape task train accuracy: 87.19262295081968
opening task train accuracy: 31.897435897435898
strap task train accuracy: 79.21146953405018
pattern task train accuracy: 66.0010240655402
carrying task train accuracy: 92.06349206349206
charm task train accuracy: 92.09445585215606
decoration task train accuracy: 88.69565217391305


Epoch: 44
Learning rate: 0.000358
color task train accuracy: 79.72350230414746
shape task train accuracy: 86.89866939611053
opening task train accuracy: 31.94871794871795
strap task train accuracy: 79.32446264073695
pattern task train accuracy: 60.55327868852459
carrying task train accuracy: 92.20912352639672
charm task train accuracy: 92.37071172555044
decoration task train accuracy: 89.89225243714726


Epoch: 45
Learning rate: 0.000341
color task train accuracy: 80.19497178040021
shape task train accuracy: 89.24731182795699
opening task train accuracy: 31.985670419651996
strap task t

Epoch: 64
Learning rate: 0.000215
color task train accuracy: 86.14674191893279
shape task train accuracy: 95.8546571136131
opening task train accuracy: 31.985670419651996
strap task train accuracy: 83.96516393442623
pattern task train accuracy: 66.88877498718607
carrying task train accuracy: 97.59344598054275
charm task train accuracy: 97.3347001537673
decoration task train accuracy: 96.36270491803278


Epoch: 65
Learning rate: 0.000204
color task train accuracy: 86.5847414234511
shape task train accuracy: 96.875
opening task train accuracy: 31.983598154792414
strap task train accuracy: 84.1107124551512
pattern task train accuracy: 67.02508960573476
carrying task train accuracy: 98.05427547363031
charm task train accuracy: 97.3374295954941
decoration task train accuracy: 96.97435897435898


Epoch: 66
Learning rate: 0.000204
color task train accuracy: 85.66308243727599
shape task train accuracy: 96.875
opening task train accuracy: 32.05128205128205
strap task train accuracy: 84.26447975

Epoch: 85
Learning rate: 0.000129
color task train accuracy: 88.57581967213115
shape task train accuracy: 98.71926229508196
opening task train accuracy: 31.899641577060933
strap task train accuracy: 85.0922131147541
pattern task train accuracy: 70.32291132752435
carrying task train accuracy: 99.02763561924259
charm task train accuracy: 99.23076923076923
decoration task train accuracy: 98.82172131147541


Epoch: 86
Learning rate: 0.000122
color task train accuracy: 88.9344262295082
shape task train accuracy: 98.71860584315736
opening task train accuracy: 31.95084485407066
strap task train accuracy: 84.98975409836066
pattern task train accuracy: 73.29923273657289
carrying task train accuracy: 99.38429964084146
charm task train accuracy: 99.18032786885246
decoration task train accuracy: 99.48770491803278


Epoch: 87
Learning rate: 0.000122
color task train accuracy: 89.44672131147541
shape task train accuracy: 98.46390168970814
opening task train accuracy: 32.00204813108039
strap task tra

...

Epoch: 274
Learning rate: 0.000001
color task train accuracy: 90.15384615384615
shape task train accuracy: 99.64194373401534
opening task train accuracy: 32.01844262295082
strap task train accuracy: 85.49461814454126
pattern task train accuracy: 72.21937467965147
carrying task train accuracy: 100.0
charm task train accuracy: 100.10251153254741
decoration task train accuracy: 99.89759344598055


Epoch: 275
Learning rate: 0.000001
color task train accuracy: 90.06147540983606
shape task train accuracy: 99.53869810353665
opening task train accuracy: 31.934493346980553
strap task train accuracy: 85.48717948717949
pattern task train accuracy: 72.41555783009211
carrying task train accuracy: 99.89754098360656
charm task train accuracy: 100.1025641025641
decoration task train accuracy: 99.84639016897081


Epoch: 276
Learning rate: 0.000001
color task train accuracy: 90.25141097998974
shape task train accuracy: 99.53846153846155
opening task train accuracy: 31.93234238851871
strap task train acc

Epoch: 295
Learning rate: 0.000001
color task train accuracy: 90.20010261672653
shape task train accuracy: 99.53869810353665
opening task train accuracy: 31.965110312981015
strap task train accuracy: 85.50947260624679
pattern task train accuracy: 72.6806765761148
carrying task train accuracy: 99.94882292732856
charm task train accuracy: 99.89769820971867
decoration task train accuracy: 100.05117707267144


Epoch: 296
Learning rate: 0.000001
color task train accuracy: 90.12282497441146
shape task train accuracy: 99.48717948717949
opening task train accuracy: 32.01844262295082
strap task train accuracy: 85.39210661199384
pattern task train accuracy: 71.67008196721312
carrying task train accuracy: 99.94871794871794
charm task train accuracy: 99.79529170931423
decoration task train accuracy: 100.0


Epoch: 297
Learning rate: 0.000001
color task train accuracy: 90.21516393442623
shape task train accuracy: 99.6417604912999
opening task train accuracy: 31.985670419651996
strap task train accu

Epoch: 316
Learning rate: 0.000000
color task train accuracy: 90.22517911975434
shape task train accuracy: 99.48744233726295
opening task train accuracy: 31.915983606557376
strap task train accuracy: 85.40706605222735
pattern task train accuracy: 70.732957457714
carrying task train accuracy: 100.0
charm task train accuracy: 99.94879672299028
decoration task train accuracy: 100.15384615384615


Epoch: 317
Learning rate: 0.000000
color task train accuracy: 90.17400204708291
shape task train accuracy: 99.53846153846155
opening task train accuracy: 31.967213114754102
strap task train accuracy: 85.40706605222735
pattern task train accuracy: 71.13613101330604
carrying task train accuracy: 100.0
charm task train accuracy: 100.1025641025641
decoration task train accuracy: 99.89748846745259


Epoch: 318
Learning rate: 0.000000
color task train accuracy: 90.26639344262296
shape task train accuracy: 99.38524590163934
opening task train accuracy: 32.0
strap task train accuracy: 85.41453428863869
p

Epoch: 337
Learning rate: 0.000000
color task train accuracy: 90.27137736815156
shape task train accuracy: 99.48744233726295
opening task train accuracy: 31.915983606557376
strap task train accuracy: 85.37084398976982
pattern task train accuracy: 71.7948717948718
carrying task train accuracy: 100.0
charm task train accuracy: 100.0
decoration task train accuracy: 100.05125576627371


Epoch: 338
Learning rate: 0.000000
color task train accuracy: 90.2763561924258
shape task train accuracy: 99.5389344262295
opening task train accuracy: 32.00204813108039
strap task train accuracy: 85.30465949820788
pattern task train accuracy: 72.13955874807594
carrying task train accuracy: 99.94879672299028
charm task train accuracy: 100.05122950819671
decoration task train accuracy: 100.1025641025641


Epoch: 339
Learning rate: 0.000000
color task train accuracy: 90.26639344262296
shape task train accuracy: 99.53917050691244
opening task train accuracy: 32.00204813108039
strap task train accuracy: 85.3333

Epoch: 358
Learning rate: 0.000000
color task train accuracy: 90.37871033776868
shape task train accuracy: 99.59016393442623
opening task train accuracy: 32.01844262295082
strap task train accuracy: 85.49461814454126
pattern task train accuracy: 73.23076923076923
carrying task train accuracy: 100.0
charm task train accuracy: 99.84639016897081
decoration task train accuracy: 100.0


Epoch: 359
Learning rate: 0.000000
color task train accuracy: 90.26639344262296
shape task train accuracy: 99.43647540983606
opening task train accuracy: 31.985670419651996
strap task train accuracy: 85.34836065573771
pattern task train accuracy: 72.13955874807594
carrying task train accuracy: 99.8974358974359
charm task train accuracy: 99.79529170931423
decoration task train accuracy: 100.05120327700973


Epoch: 360
Learning rate: 0.000000
color task train accuracy: 90.12282497441146
shape task train accuracy: 99.58974358974359
opening task train accuracy: 31.934493346980553
strap task train accuracy: 85.54